# Como Ajustar LLMs com Adaptadores LoRA usando Hugging Face TRL

Este notebook demonstra como ajustar eficientemente grandes modelos de linguagem usando adaptadores LoRA (Adaptação de Baixa Classificação). LoRA é uma técnica de ajuste fino eficiente em termos de parâmetros que:
- Congela os pesos do modelo pré-treinado
- Adiciona pequenas matrizes treináveis de decomposição de classificação às camadas de atenção
- Normalmente reduz os parâmetros treináveis em ~90%
- Mantém o desempenho do modelo sendo eficiente em termos de memória

Vamos abordar:
1. Configurar o ambiente de desenvolvimento e a configuração do LoRA
2. Criar e preparar o conjunto de dados para o treinamento do adaptador
3. Ajustar o modelo usando `trl` e `SFTTrainer` com adaptadores LoRA
4. Testar o modelo e mesclar adaptadores (opcional)


## 1. Configurar o ambiente de desenvolvimento

Nosso primeiro passo é instalar os módulos do Hugging Face e o PyTorch, incluindo trl, transformers e datasets. Se você ainda não ouviu falar do trl, não se preocupe. É uma nova biblioteca baseada em transformers e datasets, que facilita o ajuste fino, o RLHF (Reinforcement Learning from Human Feedback) e o alinhamento de modelos de linguagem abertos (open LLMs).


In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

## 2. Carregue o conjunto de dados

In [13]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. Ajustar uma LLM usando `trl` e o `SFTTrainer` com o LoRA

O [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) do `trl` oferece integração com adaptadores LoRA através do módulo [PEFT](https://huggingface.co/docs/peft/en/index). As principais vantagens dessa configuração incluem:

1. **Eficiência de Memória**: 
   - Apenas os parâmetros dos adaptadores são armazenados na memória da GPU
   - Os pesos do modelo base permanecem congelados e podem ser carregados em precisão reduzida

   - Enables fine-tuning of large models on consumer GPUs

2. **Recursos de Treinamento**:
   - Integração nativa PEFT/LoRA com configuração mínima
   - Suporte ao QLoRA (LoRA Quantizado) para uma eficiência de memória ainda melhor

3. **Gerenciamento de Adaptadores**:
   - Salvamento de pesos dos adaptadores durante checkpoints
   - Recursos para mesclar adaptadores de volta ao modelo base

Usaremos LoRA em nosso exemplo, que combina LoRA com quantização de 4 bits para reduzir ainda mais o uso de memória sem sacrificar o desempenho. A configuração requer apenas algumas etapas:
1. Definir a configuração do LoRA (rank, alpha, dropout)
2. Criar o SFTTrainer com configuração PEFT
3. Treinar e salvar os pesos dos adaptadores


In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

O `SFTTrainer` oferece uma integração nativa com o `peft`, tornando extremamente fácil ajustar eficientemente LLMs, como com o LoRA, por exemplo. Precisamos apenas criar nosso `LoraConfig` e fornecê-lo ao trainer.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'> 
    <h2 style='margin: 0;color:blue'>Exercício: Definir parâmetros LoRA para ajuste fino</h2> 
    <p>Escolha um conjunto de dados do hub do Hugging Face e ajuste um modelo com ele.</p> 
    <p><b>Níveis de Dificuldade</b></p> 
    <p>🐢 Use os parâmetros gerais para um ajuste fino arbitrário.</p> 
    <p>🐕 Ajuste os parâmetros e revise em pesos & biases.</p> 
    <p>🦁 Ajuste os parâmetros e mostre a mudança nos resultados de inferência.</p> 
</div>

In [ ]:
from peft import LoraConfig

# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 6
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 8
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

Antes de iniciarmos nosso treinamento, precisamos definir os hiperparâmetros (`TrainingArguments`) que queremos usar.


In [ ]:
# Training configuration
# Hyperparameters based on QLoRA paper recommendations
args = SFTConfig(
    # Output settings
    output_dir=finetune_name,  # Directory to save model checkpoints
    # Training duration
    num_train_epochs=1,  # Number of training epochs
    # Batch size settings
    per_device_train_batch_size=2,  # Batch size per GPU
    gradient_accumulation_steps=2,  # Accumulate gradients for larger effective batch
    # Memory optimization
    gradient_checkpointing=True,  # Trade compute for memory savings
    # Optimizer settings
    optim="adamw_torch_fused",  # Use fused AdamW for efficiency
    learning_rate=2e-4,  # Learning rate (QLoRA paper)
    max_grad_norm=0.3,  # Gradient clipping threshold
    # Learning rate schedule
    warmup_ratio=0.03,  # Portion of steps for warmup
    lr_scheduler_type="constant",  # Keep learning rate constant after warmup
    # Logging and saving
    logging_steps=10,  # Log metrics every N steps
    save_strategy="epoch",  # Save checkpoint every epoch
    # Precision settings
    bf16=True,  # Use bfloat16 precision
    # Integration settings
    push_to_hub=False,  # Don't push to HuggingFace Hub
    report_to=None,  # Disable external logging
)

Agora temos todos os pedaços que precisamos para criar nosso `SFTTrainer` para então começar a treinar nosso modelo.

In [ ]:
max_seq_length = 1512  # max sequence length for model and packing of the dataset

# Create SFTTrainer with LoRA configuration
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # LoRA configuration
    max_seq_length=max_seq_length,  # Maximum sequence length
    tokenizer=tokenizer,
    packing=True,  # Enable input packing for efficiency
    dataset_kwargs={
        "add_special_tokens": False,  # Special tokens handled by template
        "append_concat_token": False,  # No additional separator needed
    },
)

Comece treinando nosso modelo chamando a função `train ()` em nossa instância de `Trainer`. Isso iniciará o loop de treinamento e treinará nosso modelo por 3 épocas. Como estamos usando um método PEFT, salvaremos apenas os pesos do modelo adaptado e não o do modelo completo.

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

O treinamento com Flash Attention por 3 épocas com um conjunto de dados de 15 mil amostras levou 4:14:36 ​​em um `g5.2xlarge`. A instância custa `1,21 dólares/h`, o que nos leva a um custo total de apenas ~`5,3 dólares`.


### Mesclar Adaptador LoRA no Modelo Original

Ao usar LoRA, treinamos apenas os pesos do adaptador mantendo o modelo base congelado. Durante o treinamento, salvamos apenas esses pesos de adaptador leves (~2-10MB) em vez de uma cópia completa do modelo. No entanto, para implantação, você talvez queira mesclar os adaptadores de volta ao modelo base para:

1. **Implantação Simplificada**: Arquivo de modelo único em vez de modelo base + adaptadores
2. **Velocidade de Inferência**: Sem sobrecarga de computação de adaptadores
3. **Compatibilidade de Framework**: Melhor compatibilidade com serving frameworks 


In [ ]:
from peft import AutoPeftModelForCausalLM


# Load PEFT model on CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. Testar Modelo e Executar Inferência

Após a conclusão do treinamento, queremos testar nosso modelo. Vamos carregar diferentes amostras do conjunto de dados original e avaliar o modelo baseado nessas amostras, usando um loop simples e a precisão como nossa métrica.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercício Bônus: Carregar Adaptador LoRA</h2>
    <p>Use o que você aprendeu no caderno de exemplo para carregar seu adaptador LoRA treinado para inferência.</p> 
</div>

In [30]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

Vamos testar uns exemplos de prompts e ver como o modelo reage.

In [34]:
prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt) :].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)